In [63]:
import os
import re
import aocd
import string
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
from aocd import submit


#Part 1
data = aocd.get_data(day=12, year=2023).split('\n')

In [64]:
data = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""".split('\n')

In [65]:



def is_hash_at(i, placement, contig_damaged, contig_damaged_cumsum):

    end = np.array(placement) + contig_damaged_cumsum
    start = end - contig_damaged
    #print(start,end)
    for j in range(len(end)):
        if end[j] > i:
            #print(i,end[j],start[j])
            if start[j] <= i:
                return True
            else:
                return False


In [16]:
# part 1

_sum = 0
for line in tqdm(data):
    print(line)
    condition, contig_damaged = line.split(' ')
    #print('\nLine:',condition, contig_damaged)
    contig_damaged = contig_damaged.split(',')
    contig_damaged = [int(x) for x in contig_damaged]
        
    # replace dots next to eachother with single dot.
    condition = re.sub('\.+', '.', condition)
    condition = re.sub('^\.+','',condition)
    condition = re.sub('\.+$','',condition)
    
    total = len(condition)
    no_damaged = sum(contig_damaged)
    no_undamaged = total - no_damaged
    
    no_known_damaged = len([x for x in condition if x == '#'])
    
    no_unknown_damaged = no_damaged - no_known_damaged
    no_unknown = len([x for x in condition if x == '?'])
    no_unknown_undamaged = no_unknown - no_unknown_damaged
    
    possible_placements = list(range(no_undamaged + 1))

    possible_placements = list(itertools.combinations(possible_placements,len(contig_damaged)))
    print(len(possible_placements))
    contig_damaged_cumsum = np.cumsum(contig_damaged)
    for i in range(len(condition)):
        char = condition[i]
        #print(char)
        if char == '?':
            pass
        elif char == '.':
            for placement in list(possible_placements):
                if is_hash_at(i, placement, contig_damaged, contig_damaged_cumsum):
                    possible_placements.remove(placement)
        elif char == '#':
            for placement in list(possible_placements):
                #print(i, placement)
                if not is_hash_at(i, placement, contig_damaged, contig_damaged_cumsum):
                    #print('deleting...', placement)
                    possible_placements.remove(placement)
    
    print(len(possible_placements))
    _sum += len(possible_placements)
    break
_sum

  0%|          | 0/6 [00:00<?, ?it/s]

???.### 1,1,3
1
1


1

In [67]:
# Part 2

# based on https://github.com/clrfl/AdventOfCode2023/blob/master/12/explanation.ipynb

data = aocd.get_data(day=12, year=2023).split('\n')


def move_forward(values, counts, possible_transitions):
    new_counts = {key: 0 for key in counts}
    for key in counts:
        transitions = possible_transitions[key]
        for t in transitions:
            if t[0] in values:
                new_counts[t] += counts[key]
    return new_counts

def transition_to_dot(counts, possible_transitions):
    return move_forward(['.'], counts, possible_transitions)

def transition_to_hash(counts, possible_transitions):
    return move_forward(['#'], counts, possible_transitions)

def transition_to_dot_or_hash(counts, possible_transitions):
    return move_forward(['.','#'], counts, possible_transitions)

_sum = 0
for line in tqdm(data):
    condition, contig_damaged = line.split(' ')
    #print('\nLine:',condition, contig_damaged)
    contig_damaged = contig_damaged.split(',')
    contig_damaged = [int(x) for x in contig_damaged]

    
    condition = ((condition + '?')*5)[:-1]
    contig_damaged = contig_damaged*5
    
    possible_states = []
    possible_transitions = {}
    counts = {}
    damaged_part_no = 0
    no_damaged_groups = len(contig_damaged)
    for group_i in range(no_damaged_groups):
        possible_states += [f'._{group_i}']
        possible_transitions[f'._{group_i}'] = [f'._{group_i}', f'#_{damaged_part_no}']
        counts[f'._{group_i}'] = 0
        
        no_damaged_in_group_i = contig_damaged[group_i]
        for k in range(no_damaged_in_group_i):
            possible_states += [f'#_{damaged_part_no}']
            
            if k == no_damaged_in_group_i - 1:
                possible_transitions[f'#_{damaged_part_no}'] = [f'._{group_i + 1}']
            else:
                possible_transitions[f'#_{damaged_part_no}'] = [f'#_{damaged_part_no + 1}']
            
            counts[f'#_{damaged_part_no}'] = 0
            
            damaged_part_no += 1
    end_states = [f'._{group_i+1}', f'#_{damaged_part_no - 1}']
    possible_states += [f'._{group_i+1}']
    possible_transitions[f'._{group_i + 1}'] = [f'._{group_i +1 }']
    counts[f'._{group_i + 1}'] = 0
            

    
    
    #print('\nLine',condition, contig_damaged)
    #print(' Possible States: ', possible_states)
    #print(' Possible Transitions: ', possible_transitions)
    #print(' End States: ', end_states)
    
    counts['._0'] = 1
    l = 0
    for char in condition:
        if l == 10:
            pass#break
        l += 1
        #print('\nBefore: ',char, {k:v for k,v in counts.items() if v > 0})
        if char == '?':
            counts = transition_to_dot_or_hash(counts, possible_transitions)
        elif char == '.':
            counts = transition_to_dot(counts, possible_transitions)
        else:
            counts = transition_to_hash(counts, possible_transitions)
        #print('After:    ',{k:v for k,v in counts.items() if v > 0})
    no_possible_ends = sum([counts[end_state] for end_state in end_states])
    _sum += no_possible_ends
    print('Number possible ends', no_possible_ends)
_sum

  0%|          | 0/1000 [00:00<?, ?it/s]


Line ??#.????#???.#?# [1, 5, 1, 1, 1]
 Possible States:  ['._0', '#_0', '._1', '#_1', '#_2', '#_3', '#_4', '#_5', '._2', '#_6', '._3', '#_7', '._4', '#_8', '._5']
 Possible Transitions:  {'._0': ['._0', '#_0'], '#_0': ['._1'], '._1': ['._1', '#_1'], '#_1': ['#_2'], '#_2': ['#_3'], '#_3': ['#_4'], '#_4': ['#_5'], '#_5': ['._2'], '._2': ['._2', '#_6'], '#_6': ['._3'], '._3': ['._3', '#_7'], '#_7': ['._4'], '._4': ['._4', '#_8'], '#_8': ['._5'], '._5': ['._5']}
 End States:  ['._5', '#_8']
Number possible ends 3


3